In [1]:
import tensorflow as tf
import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange

In [2]:
##intergrating the text set
##here is a small text sample
##if you want to use good embedding, please try to download wikipedia
f=open("sis/annotation.txt")
text=f.readlines()
words=[]
dic=set()
for line in text:
    word=line.split()
    for w in word:
        if w.isalnum():
            dic.add(w)
            words.append(w)
f.close()

In [3]:
f=open("caption/annotation.txt")
text=f.readlines()
for line in text:
    word=line.split()
    for w in word:
        if w.isalnum():
            dic.add(w)
            words.append(w)
f.close()

In [4]:
f=open("annotations/captions.txt")
text=f.readlines()
for line in text:
    word=line.split()
    for w in word:
        w="".join(l for l in w if l not in {',','.','!','?','\"','(',')','\''})
        if len(w)>0:
            dic.add(w)
            words.append(w)
f.close()

In [5]:
vocabulary_size=len(dic)

In [6]:
vocabulary_size

37119

In [7]:
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))

In [8]:
##dataset construction
import collections
import math
def build_dataset(words, vocabulary_size):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

In [9]:
## Build the dictionary and replace rare words with UNK token.
data, count, dictionary, reverse_dictionary = build_dataset(words, vocabulary_size)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])
data_index = 0

Most common words (+UNK) [['UNK', 1], ('a', 275859), ('the', 210048), ('to', 87244), ('and', 80056)]
Sample data [50, 3363, 235, 7, 212, 9, 24, 3, 61, 4028] ['our', 'landmark', 'tree', 'in', 'town', 'was', 'about', 'to', 'be', 'destroyed']


In [10]:
##generate batch for training
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
        # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [11]:
##Build and train a skip-gram model.

batch_size = 16
embedding_size = 100  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                        biases=nce_biases,
                         labels=train_labels,
                         inputs=embed,
                         num_sampled=num_sampled,
                         num_classes=vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()

In [12]:
#Begin training.
num_steps = 200001

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print("Initialized")

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = "Nearest to %s:" % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  251.863830566
Nearest to that: sigils, caster, faring, chun, fprest, basically, trustees, saphire,
Nearest to some: 747, jonah, adrift, greets, era, biplane, joystick, mighty,
Nearest to see: facepaint, atlas, menus, woozy, rw, richest, baseline, permission,
Nearest to the: gong, solutions, badmitton, hungout, brodcosting, storybooks, ufc, coursework,
Nearest to while: bedridden, papered, supercell, installments, lump, searches, pleaded, repeating,
Nearest to water: reviews, call, miserables, 6am, gaurds, goup, siding, filth,
Nearest to everyone: unamerican, keytar, compose, incorrect, bridges, stick, frazzled, deliveries,
Nearest to talking: dwellers, invoices, squaring, refinished, malt, wilkes, vesel, pirate,
Nearest to food: uploaded, lavene, movable, doctors, deeply, sons, replicas, geodesic,
Nearest to had: realize, latches, casualness, borrowing, sadness, fossmeetnitc, rockers, recreations,
Nearest to went: 5505, difernt, regaining, loews, 

In [15]:
#Visualize the embeddings using PCA
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
def plot_with_labels(low_dim_embs, labels, filename='pca.png'):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
        plt.savefig(filename)


In [19]:
##plot a random set of terms
plot_only = 10000
pca=PCA(n_components=2)
x=pca.fit(final_embeddings[:plot_only, :])
y=x.transform(final_embeddings[:plot_only, :])
labels = [reverse_dictionary[i] for i in xrange(plot_only)]
plot_with_labels(y, labels)

In [22]:
##save embeddings
np.save("word_vector.npy",final_embeddings)
np.save("dictionary.npy",dictionary)
np.save("reverse_dictionary.npy",reverse_dictionary)